In [1]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.autonotebook import tqdm
import pandas as pd




C:\Users\jeonguihyeong\PycharmProjects\quotation\venv\lib\site-packages\mxnet\optimizer\optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))


In [3]:
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()


using cached model
using cached model


In [4]:

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.device(0))
print(torch.cuda.current_device())

True
GeForce GTX 1650 Ti
0


In [2]:
title_data = pd.read_csv('./title.csv.')
title_data.head()
title_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14604 entries, 0 to 14603
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   14604 non-null  object
 1   label   14604 non-null  object
dtypes: object(2)
memory usage: 228.3+ KB


In [6]:
title_data['label'].value_counts()

c    7310
a    4941
b    2353
Name: label, dtype: int64

In [8]:
title_data.sample(n=10)

,title,label
3048,"시진핑 ""아프간 문제 해결 위해 건설적 역할 할 것""",a
4369,"KCGI ""임시주총 열자""..한진칼 이사회 결정은?",a
12393,"두 번째 충북소방노조 출범…""존중 받는 일터 만들 것""",c
1290,"대검 ""공수처 서류, 우편 접수 거절은 사실과 달라""",a
2066,"내장사 대웅전 화재원인은 방화…경찰 ""승려 현행범 체포""",a
3874,"채희봉 ""월성1호기 조기폐쇄·가동 중단 당연 조치""",a
4294,"APEC참석한 文대통령 ""기업인 등 필수 인력 이동 방안 협의해야""",a
10388,"""모든 이에게 모든 것"" 남기고…정진석 추기경 영면",c
13,"리서치알음 ""미래 성장성 갖춘 기업을 찾아라""",a
2949,"[속보] 당국 ""셀트리온 항체 치료제 '카파형' 바이러스에도 일부 효과""",a


In [9]:
title_data.loc[(title_data['label'] == "a"), 'label'] = 0
title_data.loc[(title_data['label'] == "b"), 'label'] = 1
title_data.loc[(title_data['label'] == "c"), 'label'] = 2


In [10]:
title_data.head(5)

,title,label
0,"[단독] ""단체장과 친한데"" 해결사 역할, 수수료 챙긴 법인 대표",0
1,"1호가' 박휘순, 17살 연하 아내 천예지 ""새엄마 아닌 초혼""",0
2,"인니 침몰 잠수함 세 동강 난 채 발견...""탑승자 53명 전원 사망""",0
3,"인도 뉴델리 봉쇄령 1주 연장...""코로나 진정 기미 안 보여""",0
4,"세 동강 난 인니 침몰 잠수함…""탑승자 53명 전원사망""",0


In [11]:
title_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14604 entries, 0 to 14603
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   14604 non-null  object
 1   label   14604 non-null  object
dtypes: object(2)
memory usage: 228.3+ KB


In [12]:
title_list = []
for title, label in zip(title_data['title'], title_data['label'])  :
    data = []
    data.append(title)
    data.append(str(label))

    title_list.append(data)

In [13]:
title_list[0]

['[단독] "단체장과 친한데" 해결사 역할, 수수료 챙긴 법인 대표', '0']

In [14]:
from sklearn.model_selection import train_test_split
                                                         
title_train, title_test = train_test_split(title_list, test_size=0.2, random_state=0)

In [15]:
print(len(title_train))
print(len(title_test))
title_train[0]

11683
2921


['권영세, 당대표 불출마 "국민의힘, 환상 깨어나 현실 직시해야"', '2']

In [16]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [17]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [18]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
print('------------------')
train = BERTDataset(title_train, 0, 1, tok, max_len, True, False)
test = BERTDataset(title_test, 0, 1, tok, max_len, True, False)

using cached model
------------------


In [19]:
train[0]

(array([   2, 1170, 6951, 6579,  517,   46, 1618, 5822, 2485, 7468, 6141,
         518, 1148, 8000,  517,   46, 5136, 6527, 1342, 6855, 5655, 5059,
        4349, 6705, 7852,  518,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1]),
 array(27),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 2)

In [20]:
train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=0)

In [21]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [22]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [23]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/183 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 4.00 GiB total capacity; 2.61 GiB already allocated; 35.70 MiB free; 2.67 GiB reserved in total by PyTorch)

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
          
            logits=i
            logits = np.round(logits.detach().cpu().numpy(),2)

            print(logits)
            if np.argmax(logits) == 0:
                test_eval.append("판단유보")
            elif np.argmax(logits) == 1:
                test_eval.append("책임회피")
            elif np.argmax(logits) == 2:
                test_eval.append("선정주의")
        print(">> 이타이틀은 판단유보 {:.2f}, 책임회피 {:.2f}, 선정주의 {:.2f} 으로 측정되어 {}유형의 따옴표입니다".format(logits[0],logits[1],logits[2],test_eval[0]))
        
        

In [ ]:
sen = '"디지털·교통인프라 협력 확대"…韓·콜롬비아 정상회담'
predict(sen)

In [ ]:
test_data = pd.read_excel('/content/test.xlsx')

In [ ]:
test_list = []
for title in test_data['title'] :
    data = []
    data.append(title)

    test_list.append(data)

In [ ]:
for i in range(len(test_list)):
  print(test_list[i][0])
  predict(test_list[i][0])
  print('\n')

In [ ]:
for i in range(len(test_list)) :
  predict(test_list[i][0])

In [ ]:
test_bert = TESTDataset(test_list, 0, tok, max_len, True, False)